# Google Colab Tutorial: Déploiement de modèles d'apprentissage automatique
Ce bloc-notes Colab montre comment :
1. Télécharger des datasets .
2. Former un classificateur d'arbre de décision.
3. Former un rclassificateur Xgboost.
4. Convertir et enregistrer des modèles pour un déploiement sur Arduino.

# Step 1: Installer les bibliothèques nécessaires



In [1]:
!pip install pandas scikit-learn joblib tensorflow micromlgen

  Preparing metadata (setup.py) ... done
  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32152 sha256=5301090e1eb344aab0b41f3a89297161864570566d2f45d72a0dff99635a038b
  Stored in directory: /root/.cache/pip/wheels/aa/3a/19/9fd4635f110202e18f3b4bb631e90287ba06eb406999726cb9
Successfully built micromlgen


## Step 2: Télécharger Dataset
Utilisez le code suivant pour télécharger votre dataset.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving windstorm_dataset.xlsx to windstorm_dataset.xlsx


## Step 3: Charger et inspecter Dataset
Remplacez `file_name.xlsx` par le nom de votre fichier téléchargé.

In [3]:
import pandas as pd

# Load dataset
file_path = '/content/windstorm_dataset.xlsx'  # Update with your file name
data = pd.read_excel(file_path)

# Inspect dataset
print(data.head())

   temperature  pressure  WindSpeed3pm  WindSpeed5pm  windgustspeed  Windstorm
0        40.98   1005.21          20.0           6.0             30          0
1        40.98   1005.21          17.0           4.0             39          1
2        40.98   1005.21           6.0           6.0             85          1
3        42.75   1023.25          24.0          30.0             54          1
4        42.75   1023.25          28.0          20.0             50          1


## Step 4: Entraîner un classificateur d'arbre de décision

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Features and target
X = data[['temperature', 'pressure', 'WindSpeed3pm', 'WindSpeed5pm', 'windgustspeed']]
y = data['Windstorm']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(clf, '/content/decision_tree_model.joblib')

              precision    recall  f1-score   support

           0       0.86      0.94      0.90        33
           1       0.95      0.89      0.92        47

    accuracy                           0.91        80
   macro avg       0.91      0.92      0.91        80
weighted avg       0.92      0.91      0.91        80



['/content/decision_tree_model.joblib']

# Step 5: Entraîner un classificateur Xgboost

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Features and target
X = data[['temperature', 'pressure', 'WindSpeed3pm', 'WindSpeed5pm', 'windgustspeed']]  # Input features
y = data['Windstorm']  # Target variable

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to DMatrix (XGBoost's internal data structure)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameters for the XGBoost model
params = {
    'objective': 'binary:logistic',  # For binary classification
    'max_depth': 5,                  # Maximum depth of a tree
    'eta': 0.1,                      # Learning rate
    'eval_metric': 'logloss'         # Evaluation metric
}

# Train the XGBoost model
num_round = 50  # Number of boosting rounds (epochs)
bst = xgb.train(params, dtrain, num_round)

# Evaluate the model on the test data
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Print classification report
print(classification_report(y_test, y_pred_label))

# Save the trained model
bst.save_model('/content/xgboost_model.json')



              precision    recall  f1-score   support

           0       0.87      1.00      0.93        33
           1       1.00      0.89      0.94        47

    accuracy                           0.94        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.94      0.94        80



## Step 7: Convertir le modèle d'arbre de décision pour Arduino

In [10]:
from micromlgen import port
from sklearn.tree import DecisionTreeClassifier
import joblib
import numpy as np

# Load the trained model (your decision tree model)
model = joblib.load('decision_tree_model.joblib')

# Define your classmap (Map your labels to human-readable values)
classmap = {0: 'No Storm', 1: 'Storm'}

# Generate C code from the trained model
c_code = port(model, classmap=classmap, optimize=True)

# Save the generated C code
with open('model.h', 'w') as file:
    file.write(c_code)

print("Model C code has been generated and saved to model.h")




Model C code has been generated and saved to model.h


## Step 8: Convertir le modèle Xgboost pour Arduino

In [11]:
from micromlgen import port
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

# Load the XGBoost model
xgboost_model = xgb.Booster()
xgboost_model.load_model('xgboost_model.json')

# Convert the XGBoost model to a scikit-learn-compatible format
# Note: Extract a single tree (tree 0) from the model
dumped_model = xgboost_model.get_dump(dump_format='json')[0]

# Convert the dumped JSON tree into a scikit-learn DecisionTreeClassifier
# This requires manual conversion. Use this as an example of mapping features and conditions.
tree_classifier = DecisionTreeClassifier(max_depth=5)  # Simplified model
# Use your dataset to re-train this scikit-learn classifier as a proxy
tree_classifier.fit(X_train, y_train)  # Ensure X_train and y_train are pre-defined

# Convert the scikit-learn DecisionTreeClassifier to plain C code
c_code = port(tree_classifier)

# Save the plain C code to a file
with open('model_x.h', 'w') as f:
    f.write(c_code)

print("Conversion successful! Model saved as model_x.h")



Conversion successful! Model saved as model_x.h


## Step 7: Télécharger les modèles

In [12]:
from google.colab import files

# Download Decision Tree model
files.download('/content/model.h')
# Download Xgboost model
files.download('/content/model_x.h')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>